In [2]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd


base_url = 'https://kwhazit.ucoz.net/trans/ff6/'
next_page = '01intro.html'
rows = []
while next_page != 'index.html': 
#     print(next_page)
    page = re.get(base_url + next_page)
    soup = BeautifulSoup(page.content, 'html.parser')
    rows.extend(soup.find_all('tr'))
    next_page = soup.find('div', class_='next').find('a').attrs['href']

In [32]:
jp = []  # japanese text
tw = []  # ted woolsey's translation
kw = []  # kwhazit's translation


for i, row in enumerate(rows):
    td = row.find_all('td')
    if len(td) not in [3, 4]:
        continue
    jp.append(td[1].text)
    tw.append(td[2].text)
    kw.append(td[3].text)

In [17]:
df = pd.DataFrame({'jp': jp, 'tw': tw, 'kw': kw})

In [19]:
df.to_csv('scripts.csv', index=False)

In [18]:
from laserembeddings import Laser

laser = Laser()

embeddings = laser.embed_sentences(df['tw'], lang='eng')